**An LSTM Based Multi-Class Email Classification Using TensorFlow,NLTK**

You can find the Dataset at `https://catalog.data.gov/dataset/consumer-complaint-database`

**Importing the Dataset**

As the dataset is very huge it seems appropriate to add it in the drive and then unmount in the colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing the Required Libraries**

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords

In [ ]:
data_set=pd.read_csv('/content/drive/MyDrive/complaints.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data_set.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2022-06-01,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Public record information inaccurate,NaN,NaN,Experian Information Solutions Inc.,NY,13027.0,Servicemember,NaN,Web,2022-06-01,In progress,Yes,NaN,5623393
1,2022-06-01,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,33990.0,NaN,Consent not provided,Web,2022-06-01,Closed with non-monetary relief,Yes,NaN,5621702
2,2022-06-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33403.0,NaN,NaN,Web,2022-06-08,In progress,Yes,NaN,5645514
3,2022-06-07,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,NaN,NaN,"Rocket Mortgage, LLC",GA,31008.0,NaN,NaN,Web,2022-06-08,Closed with explanation,Yes,NaN,5643448
4,2022-06-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",AL,35473.0,NaN,NaN,Web,2022-06-08,In progress,Yes,NaN,5644133


We will be using only 2 columns namely **Product** and **Consumer complaint Narrative** for our project, so let's export that from the dataframe.

In [ ]:
df1 = data_set[['Product', 'Consumer complaint narrative']].copy()

let's further extract data which has the **Consumer complaint Narative** as not null 

In [ ]:
df1 = df1[pd.notnull(df1['Consumer complaint narrative'])]
df1.columns = ['Product', 'Consumer_complaint']

Let's take a look at the different categories that we are going to classify our data into

In [ ]:
pd.DataFrame(df1.Product.unique()).values

array([['Credit reporting, credit repair services, or other personal consumer reports'],
       ['Credit card or prepaid card'],
       ['Debt collection'],
       ['Mortgage'],
       ['Vehicle loan or lease'],
       ['Checking or savings account'],
       ['Payday loan, title loan, or personal loan'],
       ['Student loan'],
       ['Money transfer, virtual currency, or money service'],
       ['Consumer Loan'],
       ['Bank account or service'],
       ['Payday loan'],
       ['Credit card'],
       ['Credit reporting'],
       ['Other financial service'],
       ['Money transfers'],
       ['Prepaid card'],
       ['Virtual currency']], dtype=object)

Let's make our lives Easier by changing the category names into simple ones

In [ ]:
df2.replace({'Product': 
             {'Credit reporting, credit repair services, or other personal consumer reports': 
              'Credit reporting, repair, or other', 
              'Credit reporting': 'Credit reporting, repair, or other',
             'Credit card': 'Credit card or prepaid card',
             'Prepaid card': 'Credit card or prepaid card',
             'Payday loan': 'Payday loan, title loan, or personal loan',
             'Money transfer': 'Money transfer, virtual currency, or money service',
             'Virtual currency': 'Money transfer, virtual currency, or money service'}}, 
            inplace= True)
pd.DataFrame(df2.Product.unique())

,0
0,Credit card or prepaid card
1,Debt collection
2,"Credit reporting, repair, or other"
3,Vehicle loan or lease
4,Checking or savings account
5,Mortgage
6,Student loan
7,Bank account or service
8,"Money transfer, virtual currency, or money ser..."
9,"Payday loan, title loan, or personal loan"


In [ ]:
df2['category_id'] = df2['Product'].factorize()[0]
category_id_df = df2[['Product', 'category_id']].drop_duplicates()

Stop words are the words that are commonly ignored by a search engine while performing search or querying , so let's try to remove them from our mails which wont cause any error

In [ ]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
  filtered_words=[word.lower() for word in text.split() if word not in stop]
  return " ".join(filtered_words)

In [ ]:
df2['Consumer_complaint']=df2['Consumer_complaint'].map(remove_stopwords)

In [ ]:
df2

,Unnamed: 0,Product,Consumer_complaint,category_id
0,1885501,Credit card or prepaid card,received letter mail american express attempti...,0
1,2183466,Debt collection,veteran served twice xxxx xxxx xxxx. medical c...,1
2,655475,Debt collection,portofolio recovery submitted collection repor...,1
3,2527140,"Credit reporting, repair, or other",equifax xxxx xxxx still reporting past due bal...,2
4,1893551,Debt collection,getting notification xxxx debt approximately x...,1
...,...,...,...,...
9995,1330494,"Credit reporting, repair, or other","dear might concern, name "" xxxx xxxx xxxx '', ...",2
9996,1626685,Debt collection,bought car friend back xxxx title transferred ...,1
9997,2065636,Consumer Loan,keep receiving harassing phone calls personal ...,11
9998,1276903,Debt collection,collection acount showing erc. supposing acvou...,1


Let's now count the number of unique words that are found in the mails , which can help us while designing our LSTM Model

for this we are going to use **Counter** in **collections** module

In [ ]:
#counting the frequency
from collections import Counter

def count_words(text_col):
  count=Counter()
  for text in text_col.values:
    for word in text.split():
      count[word]+=1
  return count

counted_words=count_words(df2.Consumer_complaint)

In [ ]:
uniqueWordCount=len(counted_words)

Now let's split our data into train and validation dataset

In [ ]:
train_size=int(df2.shape[0]*0.8)

In [ ]:
train_data=df2[:train_size]
validate_data=df2[train_size:]

In [ ]:
train_sentences=train_data.Consumer_complaint.to_numpy()
train_labels=train_data.category_id.to_numpy()
validate_sentences=validate_data.Consumer_complaint.to_numpy()
validate_labels=validate_data.category_id.to_numpy()

Now let's tokenize our data which is nothing but adding an ID to every word tha were found in our dataset of mails

In [ ]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=uniqueWordCount)
tokenizer.fit_on_texts(train_sentences)


In [ ]:
word_index=tokenizer.word_index

In [ ]:
train_sequence=tokenizer.texts_to_sequences(train_sentences)
validate_sequence=tokenizer.texts_to_sequences(validate_sentences)

As all the mails might not have the same number of words , lets use the pad_sequences method to make them equal (*Note: We are using zero padding here)

In [ ]:
max_length=50

In [ ]:
train_padded=tf.keras.preprocessing.sequence.pad_sequences(train_sequence,maxlen=max_length,padding='post',truncating='post')
validate_padded=tf.keras.preprocessing.sequence.pad_sequences(validate_sequence,maxlen=max_length,padding='post',truncating='post')

Now  let's model our LSTM and train it . we use 13 output layer neurons each for 1 class and a softmax activation function

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(uniqueWordCount,128,input_length=max_length))
model.add(tf.keras.layers.LSTM(128,dropout=0.1))
model.add(tf.keras.layers.Dense(13,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 128)           4499328   
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 13)                1677      
                                                                 
Total params: 4,632,589
Trainable params: 4,632,589
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss='sparse_categorical_crossentropy'
opt=tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss=loss,optimizer=opt,metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_padded,train_labels,epochs=20,validation_data=(validate_padded,validate_labels),verbose=1)

Epoch 1/20
250/250 [==============================] - 35s 132ms/step - loss: 1.4888 - accuracy: 0.4995 - val_loss: 1.2526 - val_accuracy: 0.5850
Epoch 2/20
250/250 [==============================] - 31s 123ms/step - loss: 1.0561 - accuracy: 0.6524 - val_loss: 1.0308 - val_accuracy: 0.6915
Epoch 3/20
250/250 [==============================] - 31s 124ms/step - loss: 0.7947 - accuracy: 0.7524 - val_loss: 1.0571 - val_accuracy: 0.7040
Epoch 4/20
250/250 [==============================] - 31s 124ms/step - loss: 0.6380 - accuracy: 0.8018 - val_loss: 0.9540 - val_accuracy: 0.7140
Epoch 5/20
250/250 [==============================] - 31s 124ms/step - loss: 0.5568 - accuracy: 0.8315 - val_loss: 0.9828 - val_accuracy: 0.7220
Epoch 6/20
250/250 [==============================] - 31s 125ms/step - loss: 0.4377 - accuracy: 0.8641 - val_loss: 1.0942 - val_accuracy: 0.6990
Epoch 7/20
250/250 [==============================] - 31s 124ms/step - loss: 0.3860 - accuracy: 0.8826 - val_loss: 1.0548 - val_ac

In [ ]:
predictions=model.predict(train_padded)

As we can see our model has a 98% accuracy over the training set and 71% over Validation Set 

In [ ]:
prediction=[]
for pre in predictions:
  prediction.append(np.argmax(pre))

In [ ]:
mismatch_count=0
total_count=len(prediction)
for i in range(len(prediction)):
  if prediction[i]!=train_labels[i]:
    mismatch_count+=1
accuracy=((total_count-mismatch_count)/total_count)*100
print("Total mails :{} \n misclassified mails :{} \n Accuracy of Prediction : {}".format(total_count,mismatch_count,accuracy))

Total mails :8000 
 misclassified mails :85 
 Accuracy of Prediction : 98.9375
